In [1]:
from __future__ import print_function
import numpy as np
import cv2
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import glob
import os

from ShapeDetect import ShapeDetect as sd
from ArrowDetect import ArrowDetect as ad
from TextDetect_OPENCV import TextDetectAll as tda
from Diag2Graph import Diag2Graph as tg

In [2]:
def preprocessImage(image_path, resize):
    
    # load the image from disk and then preprocess it
    image = cv2.imread(image_path)
    # add white border in the original image        
    image = cv2.copyMakeBorder(image,10,10,10,10,cv2.BORDER_CONSTANT,value=[255, 255, 255]) 
    if resize == 1:
        newX, newY = image.shape[1]*1.5, image.shape[0]*1.5
    else:
        newX, newY = image.shape[1], image.shape[0]
            
    image_resize = cv2.resize(image,(int(newX),int(newY)))
        
    imgPIL = Image.open(image_path)
    imgPIL = ImageOps.expand(imgPIL, border = 10, fill = 'white')
    imgPIL = imgPIL.resize((int(newX),int(newY)), Image.ANTIALIAS)        
    imgPIL = ImageEnhance.Color(imgPIL)
    imgPIL = imgPIL.enhance(0)
    gray_im = imgPIL.convert('L') 

    gray_imcv = np.array(gray_im, dtype=np.uint8)    
    _, thresh_im = cv2.threshold(gray_imcv, 240, 255, cv2.THRESH_BINARY_INV)    
        
    return image_resize, thresh_im, gray_imcv

In [3]:
imdir_path = "Input"
op_dir = "Output"

In [4]:
print("[INFO] loading images ...")
    
for filename in glob.glob(os.path.join(imdir_path, '*png')):
    print(filename)
    im, thresh_im, gray_imcv = preprocessImage(filename, 0)
        
    shapedetector = sd()
    component = shapedetector.find_component(filename, op_dir, im, thresh_im, gray_imcv)
               
    textdetector = tda()
    text_list = textdetector.combinedTextDetect(filename, im, component)
        
    arrowdetector = ad()            
    line_connect = arrowdetector.detectLines(im, thresh_im, gray_imcv, component, text_list)
    
    graphcreator = tg()
    graphcreator.createDiag2Graph(op_dir, filename, im, thresh_im, component, text_list, line_connect)
        

[INFO] loading images ...
Input/fig1612.04928-Figure3-1.png
=======+++++++++++++++++++++=======
Component number 0 has text: Input  

Component number 0 has text: layer  

=======+++++++++++++++++++++=======
Component number 1 has text: Fully Connected  

Component number 1 has text: layer  

=======+++++++++++++++++++++=======
Component number 2 has text: Dropout  

=======+++++++++++++++++++++=======
Component number 3 has text: LSTM  

=======+++++++++++++++++++++=======
Component number 4 has text: Output Layer  

Component number 3 is connected to Component number 4 

Component number 2 is connected to Component number 3 

Component number 1 is connected to Component number 2 

Component number 1 is connected to Component number 0 

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Input/fig1604.03489-Figure5-1.png
Component 0: Position: (74 22 84 25), text: cn, confidence = 65

Component 1: Position: (179 18 230 30), te

=======+++++++++++++++++++++=======
Component number 0 has text: 5 

=======+++++++++++++++++++++=======
Component number 2 has text: 13 

=======+++++++++++++++++++++=======
Component number 3 has text: 13 

=======+++++++++++++++++++++=======
Component number 4 has text: 2713 

=======+++++++++++++++++++++=======
Component number 5 has text: 55527  

=======+++++++++++++++++++++=======
Component number 6 has text: 11 

=======+++++++++++++++++++++=======
Component number 7 has text:  

Component number 7 has text: x 

Component number 7 has text: dp 

Component number 7 has text: 3 

Component number 7 has text: 2 

=======+++++++++++++++++++++=======
Component number 8 has text: 1e2306  

Component number 8 has text: x 

=======+++++++++++++++++++++=======
Component number 9 has text: 115  

=======+++++++++++++++++++++=======
Component number 10 has text:  

=======+++++++++++++++++++++=======
Component number 11 has text: cb 

=======+++++++++++++++++++++=======
Component number 1